## Setup

### Load Data

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cpi
import datetime
import seaborn as sns

# read in data
df = pd.read_csv('cleaned_data.csv')

# list all column names without ...
pd.set_option('display.max_columns', None)
df.head()


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3258: DtypeWarning: Columns (20,32,33,41,42,43,44,45,46,47,48,49,50,106,107,112) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Reporting Railroad Name,Report Year,Accident Year,Accident Month,Other Railroad Name,Other Accident Number,Other Accident Year,Other Accident Month,Maintenance Railroad Name,Maintenance Accident Year,Maintenance Accident Month,Grade Crossing ID,Day,Date,Time,Accident Type,Hazmat Cars,Hazmat Cars Damaged,Hazmat Released Cars,Persons Evacuated,Subdivision,Division,Station,Milepost,State Abbreviation,County Name,District,Temperature,Visibility,Weather Condition,Track Type,Track Name,Track Class,Track Density,Train Direction,Equipment Type,Equipment Attended,Train Speed,Recorded Estimated Speed,Maximum Speed,Gross Tonnage,Signalization,Method of Operation,Adjunct Code 1,Adjunct Code Name 1,Adjunct Code 2,Adjunct Code Name 2,Adjunct Code 3,Adjunct Code Name 3,Remote Control Locomotive,First Car Number,First Car Position,First Car Loaded,Causing Car Initials,Causing Car Number,Causing Car Position,Causing Car Loaded,Positive Alcohol Tests,Positive Drug Tests,Passengers Transported,Head End Locomotives,Mid Train Manual Locomotives,Mid Train Remote Locomotives,Rear End Manual Locomotives,Rear End Remote Locomotives,Derailed Head End Locomotives,Derailed Mid Train Manual Locomotives,Derailed Mid Train Remote Locomotives,Derailed Rear End Manual Locomotives,Derailed Rear End Remote Locomotives,Loaded Freight Cars,Loaded Passenger Cars,Empty Freight Cars,Empty Passenger Cars,Cabooses,Derailed Loaded Freight Cars,Derailed Loaded Passenger Cars,Derailed Empty Freight Cars,Derailed Empty Passenger Cars,Derailed Cabooses,Equipment Damage Cost,Track Damage Cost,Total Damage Cost,Primary Accident Cause,Contributing Accident Cause,Accident Cause,Engineers On Duty,Firemen On Duty,Conductors On Duty,Brakemen On Duty,Hours Engineers On Duty,Minutes Engineers On Duty,Hours Conductors On Duty,Minutes Conductors On Duty,Railroad Employees Killed,Railroad Employees Injured,Passengers Killed,Passengers Injured,Others Killed,Others Injured,Persons Killed For Reporting Railroad,Persons Injured For Reporting Railroad,Total Persons Killed,Total Persons Injured,Total Killed Form 54,Total Injured Form 54,Special Study 1,Special Study 2,Latitude,Longitude,Narrative,Joint Track Type,Joint Track Class,Class,Joint CD,Reporting Railroad Company Grouping,Reporting Railroad Class,Reporting Railroad SMT Grouping,Reporting Parent Railroad Company Name,Reporting Railroad Holding Company,Other Railroad Company Grouping,Other Railroad Class,Other Railroad SMT Grouping,Other Parent Railroad Company Name,Other Railroad Holding Company,Maintenance Railroad Company Grouping,Maintenance Railroad Class,Maintenance Railroad SMT Grouping,Maintenance Parent Railroad Company Name,Maintenance Railroad Holding Company
0,Norfolk Southern Railway Company,2016.0,16.0,4.0,NaN,NaN,NaN,NaN,Norfolk Southern Railway Company,16.0,4.0,NaN,9.0,04/09/2016,5:30 AM,Derailment,0.0,0.0,0.0,0.0,GEORGIA,NaN,MACON,242.00,GA,BIBB,3.0,60.0,Dawn,Clear,Yard,EAST PULLBACK,1,0000,North,Yard/switching,Yes,6.0,Estimated,6,0.0,Not Signaled,Other Than Main Track,K,Restricted Speed or Equivalent,NaN,NaN,NaN,NaN,Not a remotely controlled operation,150060,13.0,No,NaN,NaN,0.0,NaN,0.0,0.0,No,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30.0,0.0,20.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,36200.0,0.0,41720.0,Failure to comply with restricted speed or its...,NaN,Failure to comply with restricted speed or its...,1.0,0.0,1.0,0.0,6.0,40.0,6.0,40.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,OTH,000-000-000,32.813658,-83.622785,WHILE PULLING NORTH THE GM42 FAILED TO COMPLY ...,Yard,1,CL,3.0,NaN,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company,NaN,NaN,NaN,NaN,NaN,NaN,Class 1,SMT-3 - Norfolk Southern,Norfolk Southern Railway Company,Norfolk Southern Railway Company
1,Norfolk Southern Railway Company,2016.0,16.0,4.0,NaN,NaN,NaN,NaN,Norfolk Southern Railway Company,16.0,4.0,NaN,9.0,04/09/2016,5:30 AM,Derailment,4.0,0.0,0.0,0.0,GEORGIA,NaN,MACON,242.00,GA,BIBB,3.0,60.0,Dawn,Clear,Yard,EA

In [85]:
df['Accident Cause'].value_counts()

Wide gage (due to defective or missing crossties)                     12103
Switch improperly lined                                                8592
Shoving movement, absence of man on or at leading end of movement      7247
Switch point worn or broken                                            7062
Buffing or slack action excessive, train handling                      5778
                                                                      ...  
Center pin broken or missing (LOCOMOTIVE)                                 1
Attempted Suicide (Highway-Rail Grade Crossing Accident)                  1
Center plate disengaged from truck unit/off center (LOCOMOTIVE)           1
Side frame broken (LOCOMOTIVE)                                            1
Journal fractured, cold break, previously overheated  (LOCOMOTIVE)        1
Name: Accident Cause, Length: 382, dtype: int64

### Cleaning Columns/Features We Are Using

In [43]:
model_dat = df.loc[:,
        ['Reporting Railroad Name',
        'Report Year',
        'Other Railroad Name',
        'Maintenance Railroad Name',
        'Grade Crossing ID',
        'Date',
        'Time',
        'Accident Type',
        'Hazmat Cars',
        'Hazmat Cars Damaged',
        'Hazmat Released Cars',
        'Persons Evacuated',
        'Division',
        'Station',
        'Milepost',
        'State Abbreviation',
        'County Name',
        'District',
        'Temperature',
        'Visibility',
        'Weather Condition',
        'Track Type',
        'Track Class',
        'Track Density',
        'Train Direction',
        'Equipment Type',
        'Equipment Attended',
        'Train Speed',
        'Recorded Estimated Speed',
        'Maximum Speed',
        'Gross Tonnage',
        'Signalization',
        'Method of Operation',
        'First Car Position',
        'First Car Loaded',
        'Causing Car Position',
        'Causing Car Loaded',
        'Positive Alcohol Tests',
        'Positive Drug Tests',
        'Passengers Transported',
        'Loaded Freight Cars',
        'Loaded Passenger Cars',
        'Derailed Loaded Freight Cars',
        'Derailed Loaded Passenger Cars',
        'Derailed Empty Freight Cars',
        'Derailed Empty Passenger Cars',
        'Total Damage Cost',
        'Primary Accident Cause',
        'Contributing Accident Cause',
        'Engineers On Duty',
        'Firemen On Duty',
        'Conductors On Duty',
        'Brakemen On Duty',
        'Total Persons Killed',
        'Total Persons Injured',
        'Latitude',
        'Longitude',
        'Joint Track Type',
        'Joint Track Class',
        'Class']]

#Clean and create time/cost features

# Drop all rows with NA in Report Year or Total Damage Cost
model_dat = model_dat.dropna(subset=['Report Year', 'Total Damage Cost'])

# Convert Report Year to int
model_dat['Year'] = model_dat['Report Year'].astype(int)
model_dat = model_dat.drop('Report Year', axis=1)

# Remove commas from Total Damage Cost
model_dat['Total Damage Cost'] = model_dat['Total Damage Cost'].astype(str).str.replace(',', '')

# Convert Total Damage Cost to int
model_dat['Total Damage Cost'] = model_dat['Total Damage Cost'].astype(float).astype(int)

# Adjust for inflation
model_dat['Total Damage Cost'] = model_dat.apply(lambda row: cpi.inflate(row['Total Damage Cost'], row['Year']), axis=1)

# Convert Date to Datetime
model_dat = model_dat.dropna(subset=['Date'])
model_dat['date'] = pd.to_datetime(model_dat['Date'])
model_dat = model_dat.drop('Date',axis=1)

# new time features based on datetime var/index
model_dat['dayofweek'] = model_dat['date'].dt.dayofweek.astype(int)
model_dat['month'] = model_dat['date'].dt.month.astype(int)

# set index to date
model_dat = model_dat.set_index('date')






### Prepreprocess Pipeline

In [54]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_selection import SelectPercentile, chi2
from sklearn.compose import make_column_selector as selector

def before_preprocess(data):
    num_data = data.select_dtypes(include=['float64', 'int64'])
    cat_data = data.select_dtypes(include=['object'])
    cat_data.fillna('NA', inplace=True)
    cat_data = cat_data.astype(str)
    new_dat = pd.concat([num_data,cat_data], axis=1)
    return new_dat

numeric_transformer = Pipeline(
    steps=[("imputer", SimpleImputer(strategy="mean")), ("scaler", StandardScaler())]
)
'''categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore")),
        ("selector", SelectPercentile(chi2, percentile=50)),
    ]
)'''

preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, selector(dtype_exclude=object)),
        ("cat", OneHotEncoder(handle_unknown="ignore"), selector(dtype_include=object)),
    ]
)

## Modeling Data

### Train/Test Split

In [56]:
from random import sample
from sklearn.model_selection import train_test_split

# rows after conrail disbands
testing_dat = model_dat[model_dat['Year']>1999]
testing_dat = before_preprocess(testing_dat)

X = testing_dat.loc[:,testing_dat.columns!="Total Damage Cost"]
Y = testing_dat["Total Damage Cost"].astype(int)
print(Y.head())

# creating testing and training data
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size=.3)

print("X-train shape: ", x_train.shape)
print("Y-train shape: ", y_train.shape)
print("X-test shape: ", x_test.shape)
print("Y-test shape: ", y_test.shape)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/pandas/core/frame.py:4327: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


date
2016-04-09    50871
2016-04-09    50871
2016-04-16    37713
2016-04-16    37713
2018-01-31    19812
Name: Total Damage Cost, dtype: int64
X-train shape:  (49776, 60)
Y-train shape:  (49776,)
X-test shape:  (21333, 60)
Y-test shape:  (21333,)


### Using GradientBooster

In [64]:
from sklearn.ensemble import GradientBoostingRegressor

gbreg = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", GradientBoostingRegressor(verbose=True,
                                                                                    #validation_fraction=0.2,
                                                                                    #n_iter_no_change=5,
                                                                                    #tol=0.05,
                                                                                    random_state=0,))]
)

gbreg.fit(x_train,y_train)



      Iter       Train Loss   Remaining Time 
         1 373151209265.8109           42.98s
         2 346383777795.4160           42.94s
         3 323159596149.3051           40.48s
         4 302946733287.9346           38.35s
         5 286886935275.3272           37.23s
         6 270620872379.0144           36.34s
         7 256402525035.9323           41.10s
         8 245601439318.1490           39.56s
         9 235023321627.6828           38.36s
        10 225626495573.2248           37.28s
        20 170686178760.0637           29.73s
        30 147219988635.8222           25.73s
        40 133522121094.8093           21.44s
        50 124037832022.0737           17.85s
        60 117722749367.5349           15.73s
        70 112486177349.4456           11.68s
        80 107890450448.8925            7.75s
        90 104298609352.0457            3.82s
       100 99856451759.9262            0.00s


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x14c6649e8>),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x14c68c4e0>)])),
                ('classifier',
                 GradientBoostingRegressor(random_state=0, verbose=True))])

In [65]:
gbreg.score(x_test,y_test)

0.5968542973376236

In [78]:
'''importances = pd.DataFrame()
importances['feature_names'] = model_dat.columns.astype(str)
importances['feature_importance'] = gbreg['classifier'].feature_importances_
importances.sort_values('feature_importance', ascending=False)'''

pd.Series(gbreg['classifier'].feature_importances_)
gbreg['classifier'].feature_names_in_

AttributeError: 'GradientBoostingRegressor' object has no attribute 'feature_names_in_'

### Random Forest Regressor

In [67]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import tree

# turn conf matrix in df for visualization
def conf_matrix_to_df(conf_matrix, target_names):
    return pd.DataFrame(conf_matrix, columns=target_names, index=target_names)

rf_pipe = Pipeline(
    steps=[("preprocessor", preprocessor), ("classifier", RandomForestRegressor(max_features='sqrt',
                                                                                verbose=True,
                                                                                n_jobs=-1))]
)

rf_pipe.fit(x_train, y_train)
#conf_mat = confusion_matrix(y_test, pred)
#conf_mat_df = conf_matrix_to_df(conf_mat,testing_dat['Accident Type'].unique())
#conf_mat_df

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.4min finished


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer()),
                                                                  ('scaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x14c6649e8>),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x14c68c4e0>)])),
                ('classifier',
                 RandomForestRegressor(max_features='sqrt', n_jobs=-1,
                                       verbose=True))])

In [68]:
pred = rf_pipe.predict(x_test)
rf_pipe.score(x_test,y_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    2.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 100 out of 100 | elapsed:    1.9s finished


0.4336961363570513

In [82]:
pd.Series(rf_pipe['classifier'].feature_importances_)
rf_pipe['classifier'].feature_names_

AttributeError: 'RandomForestRegressor' object has no attribute 'feature_names_'

### MLR 